In [1]:
import json

train_root_path = '/hpi/fs00/share/fg/rabl/strassenburg/datasets/coco/train2017'
train_annotations = '/hpi/fs00/share/fg/rabl/strassenburg/datasets/coco/annotations/instances_train2017.json'

val_root_path = '/hpi/fs00/share/fg/rabl/strassenburg/datasets/coco/val2017'
val_annotations = '/hpi/fs00/share/fg/rabl/strassenburg/datasets/coco/annotations/instances_val2017.json'

In [2]:
%matplotlib inline

In [3]:
def get_all_categories(annotations):
    ids = []
    for a in annotations:
        ids.append(a['category_id'])

    return ids


def id_to_class_index(annotations_path):
    index = {}
    with open(annotations_path, 'r') as COCO:
        js = json.loads(COCO.read())
        cats = js['categories']

        for cat in cats:
            index[cat['id']] = cat['name']

    return index


In [ ]:
train_cat_index = id_to_class_index(train_annotations)
val_cat_index = id_to_class_index(val_annotations)

In [ ]:
print(len(train_cat_index))
print(len(val_cat_index))

print(val_cat_index == train_cat_index)